In [2]:
!pip install meteostat

In [69]:
import yfinance as yf
import pandas as pd
from functools import reduce
import time

# Your tickers list
tickers = ['AUROPHARMA.NS', 'AXISBANK.NS', 'ADANIPOWER.NS', 'ADANIGREEN.NS', 'ADANIENT.NS']

# Configuration
batch_size = 50
start_date = '2020-03-21'
end_date = '2022-03-22'
max_retries = 3
delay_between_retries = 5

# Create a full calendar of business days
all_dates = pd.bdate_range(start=start_date, end=end_date)
datewise_tracker = {date: [] for date in all_dates}

def download_with_retries(ticker_batch):
    attempt = 0
    while attempt < max_retries:
        try:
            return yf.download(ticker_batch, start=start_date, end=end_date, group_by='ticker', progress=False)
        except Exception as e:
            print(f"Error downloading {ticker_batch}: {e}")
            attempt += 1
            time.sleep(delay_between_retries)
    return None

for i in range(0, len(tickers), batch_size):
    batch = tickers[i:i + batch_size]
    print(f"\n🔄 Processing batch {i // batch_size + 1} / {len(tickers) // batch_size + 1}")

    data = download_with_retries(batch)
    if data is None:
        continue

    # Normalize format for single ticker
    if len(batch) == 1:
        ticker = batch[0]
        data.columns = pd.MultiIndex.from_product([[ticker], data.columns])

    for ticker in batch:
        try:
            df = data[ticker][['Open', 'Close']].dropna()
            for date, row in df.iterrows():
                if date in datewise_tracker:
                    open_price = row['Open']
                    close_price = row['Close']
                    value = 1 + ((open_price - close_price) / open_price)
                    datewise_tracker[date].append(value)
        except Exception as e:
            print(f"⚠️ Skipping {ticker} due to: {e}")
            continue

# Now calculate final value per date
final_values = []
for date in all_dates:
    daily_values = datewise_tracker[date]
    if daily_values:
        product = reduce(lambda x, y: x * y, daily_values)
    else:
        product = None  # or 1.0 if you want to assume neutral
    final_values.append({'Date': date, 'Multiplied_Value': product})

# Create DataFrame and save
result_df = pd.DataFrame(final_values)
result_df.to_csv('multiplied_ohlc_all_dates.csv', index=False)

print("\n✅ All dates processed. Saved as 'multiplied_ohlc_all_dates.csv'")


🔄 Processing batch 1 / 1

✅ All dates processed. Saved as 'multiplied_ohlc_all_dates.csv'


In [81]:
import yfinance as yf
import pandas as pd

# Your list of tickers
tickers = ['AUROPHARMA.NS', 'AXISBANK.NS', 'ADANIPOWER.NS', 'ADANIGREEN.NS', 'ADANIENT.NS']

# Date range
start_date = "2025-01-01"
end_date = "2025-02-01"

# Store results
result = pd.DataFrame()

for ticker in tickers:
    df = yf.download(ticker, start=start_date, end=end_date, interval="1d")
    if not df.empty:
        # Calculate (Open - Close) / Open
        df[ticker] = (df['Open'] - df['Close']) / df['Open']
        result = pd.concat([result, df[[ticker]]], axis=1)

# Final sum across all companies per date
result['Sum_Per_Date'] = result.sum(axis=1)

# Show it
print(result[['Sum_Per_Date']].head())


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Price      Sum_Per_Date
Ticker                 
Date                   
2025-01-01    -0.025691
2025-01-02    -0.007946
2025-01-03     0.044478
2025-01-06     0.157542
2025-01-07    -0.049403


In [95]:
import yfinance as yf
import pandas as pd

# List of tickers
tickers = ['AUROPHARMA.NS', 'AXISBANK.NS', 'ADANIPOWER.NS', 'ADANIGREEN.NS', 'ADANIENT.NS']

# Date range
start_date = "2021-12-14"
end_date = "2022-12-14"

# DataFrame to collect results
result = pd.DataFrame()

for ticker in tickers:
    df = yf.download(ticker, start=start_date, end=end_date, interval="1d")
    if not df.empty:
        df[ticker] = (df['Open'] - df['Close']) / df['Open']
        result = pd.concat([result, df[[ticker]]], axis=1)

# Sum across all tickers for each date
result['Sum_Per_Date'] = result.sum(axis=1)

# Save to CSV
result[['Sum_Per_Date']].to_csv("sum_per_date.csv")

print("CSV file 'sum_per_date.csv' has been saved.")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

CSV file 'sum_per_date.csv' has been saved.


In [13]:
import yfinance as yf
import pandas as pd

# List of tickers
tickers = ['AUROPHARMA.NS', 'AXISBANK.NS', 'ADANIPOWER.NS', 'ADANIGREEN.NS', 'ADANIENT.NS']

# Date range
start_date = "2020-01-02"
end_date = "2021-01-03"

# DataFrame to collect results
result = pd.DataFrame()

for ticker in tickers:
    df = yf.download(ticker, start=start_date, end=end_date, interval="1d")
    if not df.empty:
        df[ticker] = (df['Open'] - df['Close']) / df['Open']
        result = pd.concat([result, df[[ticker]]], axis=1)

# Sum across all tickers for each date
result['Sum_Per_Date'] = result.sum(axis=1)

# Calculate the total sum of Sum_Per_Date column
total_sum = result['Sum_Per_Date'].sum()
print("Total sum of 'Sum_Per_Date':", total_sum)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Total sum of 'Sum_Per_Date': -1.0264522518304378


In [ ]:
import yfinance as yf
import pandas as pd
import os
import time
from tenacity import retry, wait_fixed, stop_after_attempt

# === CONFIGURATION ===
tickers = ['3MINDIA.NS', '63MOONS.NS', 'AARTIDRUGS.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABB.NS', 'ABBOTINDIA.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ACC.NS', 'ACE.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'ADSL.NS', 'ADVENZYMES.NS', 'AFFLE.NS', 'AHLUCONT.NS', 'AIAENG.NS', 'AJANTPHARM.NS', 'AJMERA.NS', 'ALEMBICLTD.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALLCARGO.NS', 'ALOKINDS.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANTRAJ.NS', 'ANUP.NS', 'APARINDS.NS', 'APLAPOLLO.NS', 'APLLTD.NS', 'APOLLO.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'ARTEMISMED.NS', 'ARVIND.NS', 'ARVINDFASN.NS', 'ASAHIINDIA.NS', 'ASALCBR.NS', 'ASHAPURMIN.NS', 'ASHOKA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTEC.NS', 'ASTERDM.NS', 'ASTRAL.NS', 'ASTRAMICRO.NS', 'ASTRAZEN.NS', 'ATGL.NS', 'ATUL.NS', 'AUBANK.NS', 'AURIONPRO.NS', 'AUROPHARMA.NS', 'AVANTIFEED.NS', 'AXISBANK.NS', 'AXISCADES.NS', 'BAJAJ-AUTO.NS', 'BAJAJELEC.NS', 'BAJAJFINSV.NS', 'BAJAJHIND.NS', 'BAJAJHLDNG.NS', 'BAJFINANCE.NS', 'BALAMINES.NS', 'BALKRISIND.NS', 'BALMLAWRIE.NS', 'BALPHARMA.NS', 'BALRAMCHIN.NS', 'BANCOINDIA.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'BASF.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BBL.NS', 'BBTC.NS', 'BDL.NS', 'BECTORFOOD.NS', 'BEL.NS', 'BEML.NS', 'BEPL.NS', 'BERGEPAINT.NS', 'BFUTILITIE.NS', 'BHARATFORG.NS', 'BHARTIARTL.NS', 'BHEL.NS', 'BIOCON.NS', 'BIRLACORPN.NS', 'BLS.NS', 'BLUEDART.NS', 'BLUESTARCO.NS', 'BOMDYEING.NS', 'BORORENEW.NS', 'BOSCHLTD.NS', 'BPCL.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'BSE.NS', 'BSOFT.NS', 'CAMLINFINE.NS', 'CAMS.NS', 'CANBK.NS', 'CANFINHOME.NS', 'CAPACITE.NS', 'CAPLIPOINT.NS', 'CARBORUNIV.NS', 'CARERATING.NS', 'CASTROLIND.NS', 'CCL.NS', 'CDSL.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CERA.NS', 'CESC.NS', 'CGCL.NS', 'CGPOWER.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHENNPETRO.NS', 'CHOLAFIN.NS', 'CHOLAHLDNG.NS', 'CIGNITITEC.NS', 'CIPLA.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COLPAL.NS', 'CONCOR.NS', 'COROMANDEL.NS', 'CREDITACC.NS', 'CRISIL.NS', 'CROMPTON.NS', 'CSBBANK.NS', 'CUB.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DATAMATICS.NS', 'DBL.NS', 'DBREALTY.NS', 'DCAL.NS', 'DCBBANK.NS', 'DCMSHRIRAM.NS', 'DCW.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DELTACORP.NS', 'DHAMPURSUG.NS', 'DHANI.NS', 'DISHTV.NS', 'DIVISLAB.NS', 'DIXON.NS', 'DLF.NS', 'DLINKINDIA.NS', 'DMART.NS', 'DONEAR.NS', 'DRREDDY.NS', 'ECLERX.NS', 'EDELWEISS.NS', 'EICHERMOT.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'EKC.NS', 'ELECON.NS', 'ELECTCAST.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'EPL.NS', 'EQUITASBNK.NS', 'ERIS.NS', 'ESCORTS.NS', 'EXCELINDUS.NS', 'EXIDEIND.NS', 'FACT.NS', 'FCL.NS', 'FEDERALBNK.NS', 'FINCABLES.NS', 'FINEORG.NS', 'FINPIPE.NS', 'FLUOROCHEM.NS', 'FORCEMOT.NS', 'FORTIS.NS', 'FSL.NS', 'GABRIEL.NS', 'GAEL.NS', 'GAIL.NS', 'GALLANTT.NS', 'GANECOS.NS', 'GANESHHOUC.NS', 'GARFIBRES.NS', 'GENESYS.NS', 'GENUSPOWER.NS', 'GEOJITFSL.NS', 'GEPIL.NS', 'GESHIP.NS', 'GHCL.NS', 'GICHSGFIN.NS', 'GICRE.NS', 'GILLETTE.NS', 'GIPCL.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'GLOBUSSPR.NS', 'GMDCLTD.NS', 'GMMPFAUDLR.NS', 'GNFC.NS', 'GODFRYPHLP.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GOKEX.NS', 'GOLDIAM.NS', 'GOODLUCK.NS', 'GPIL.NS', 'GPPL.NS', 'GPTINFRA.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAVITA.NS', 'GREAVESCOT.NS', 'GRSE.NS', 'GSFC.NS', 'GSPL.NS', 'GTLINFRA.NS', 'GUJALKALI.NS', 'GUJGASLTD.NS', 'GULFOILLUB.NS', 'HAL.NS', 'HAPPSTMNDS.NS', 'HATHWAY.NS', 'HAVELLS.NS', 'HCC.NS', 'HCG.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HEG.NS', 'HEMIPROP.NS', 'HERITGFOOD.NS', 'HEROMOTOCO.NS', 'HESTERBIO.NS', 'HFCL.NS', 'HGINFRA.NS', 'HIKAL.NS', 'HIL.NS', 'HIMATSEIDE.NS', 'HINDALCO.NS', 'HINDCOPPER.NS', 'HINDOILEXP.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'HITECH.NS', 'HONAUT.NS', 'HPL.NS', 'HSCL.NS', 'HUBTOWN.NS', 'HUDCO.NS', 'IBREALEST.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'ICIL.NS', 'IDBI.NS', 'IDEA.NS', 'IDFCFIRSTB.NS', 'IEX.NS', 'IFBIND.NS', 'IFCI.NS', 'IGARASHI.NS', 'IGL.NS', 'IIFL.NS', 'IMAGICAA.NS', 'IMFA.NS', 'INDHOTEL.NS', 'INDIAGLYCO.NS', 'INDIAMART.NS', 'INDIANB.NS', 'INDIANHUME.NS', 'INDIGO.NS', 'INDOTECH.NS', 'INDRAMEDCO.NS', 'INDUSINDBK.NS', 'INDUSTOWER.NS', 'INFIBEAM.NS', 'INFY.NS', 'INOXWIND.NS', 'INTELLECT.NS', 'IOB.NS', 'IOC.NS', 'IOLCP.NS', 'IPCALAB.NS', 'IRB.NS', 'IRCON.NS', 'IRCTC.NS', 'ITC.NS', 'ITDCEM.NS', 'J&KBANK.NS', 'JAIBALAJI.NS', 'JAICORPLTD.NS', 'JAMNAAUTO.NS', 'JASH.NS', 'JBCHEPHARM.NS', 'JBMA.NS', 'JCHAC.NS', 'JINDALSAW.NS', 'JINDALSTEL.NS', 'JINDRILL.NS', 'JINDWORLD.NS', 'JISLJALEQS.NS', 'JKCEMENT.NS', 'JKIL.NS', 'JKLAKSHMI.NS', 'JKPAPER.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JPASSOCIAT.NS', 'JPPOWER.NS', 'JSL.NS', 'JSWENERGY.NS', 'JSWHL.NS', 'JSWSTEEL.NS', 'JUBLFOOD.NS', 'JUSTDIAL.NS', 'JYOTHYLAB.NS', 'JYOTISTRUC.NS', 'KABRAEXTRU.NS', 'KAJARIACER.NS', 'KAMATHOTEL.NS', 'KAMDHENU.NS', 'KARURVYSYA.NS', 'KCP.NS', 'KDDL.NS', 'KEC.NS', 'KEI.NS', 'KELLTONTEC.NS', 'KERNEX.NS', 'KIRIINDUS.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KITEX.NS', 'KNRCON.NS', 'KOTAKBANK.NS', 'KPITTECH.NS', 'KPRMILL.NS', 'KRBL.NS', 'KSB.NS', 'KSCL.NS', 'KTKBANK.NS', 'LALPATHLAB.NS', 'LAURUSLABS.NS', 'LEMONTREE.NS', 'LICHSGFIN.NS', 'LINDEINDIA.NS', 'LT.NS', 'LTTS.NS', 'LUPIN.NS', 'LUXIND.NS', 'M&M.NS', 'M&MFIN.NS', 'MAHABANK.NS', 'MAHLIFE.NS', 'MAHSCOOTER.NS', 'MAHSEAMLES.NS', 'MANAPPURAM.NS', 'MANGCHEFER.NS', 'MANGLMCEM.NS', 'MANINDS.NS', 'MANINFRA.NS', 'MARICO.NS', 'MARINE.NS', 'MARKSANS.NS', 'MARUTI.NS', 'MASTEK.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'MCX.NS', 'METROPOLIS.NS', 'MFSL.NS', 'MGL.NS', 'MIDHANI.NS', 'MINDACORP.NS', 'MMTC.NS', 'MOIL.NS', 'MOREPENLAB.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MPSLTD.NS', 'MRF.NS', 'MRPL.NS', 'MSTCLTD.NS', 'MTNL.NS', 'MUTHOOTFIN.NS', 'NACLIND.NS', 'NAM-INDIA.NS', 'NATCOPHARM.NS', 'NATIONALUM.NS', 'NAUKRI.NS', 'NAVINFLUOR.NS', 'NAVKARCORP.NS', 'NBCC.NS', 'NCC.NS', 'NECLIFE.NS', 'NELCO.NS', 'NEOGEN.NS', 'NESCO.NS', 'NESTLEIND.NS', 'NETWORK18.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NFL.NS', 'NH.NS', 'NHPC.NS', 'NIACL.NS', 'NIITLTD.NS', 'NITINSPIN.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NOCIL.NS', 'NSIL.NS', 'NTPC.NS', 'OAL.NS', 'OBEROIRLTY.NS', 'OFSS.NS', 'OIL.NS', 'OLECTRA.NS', 'ONGC.NS', 'OPTIEMUS.NS', 'ORIENTCEM.NS', 'ORIENTELEC.NS', 'ORISSAMINE.NS', 'PAGEIND.NS', 'PAISALO.NS', 'PANACEABIO.NS', 'PARACABLES.NS', 'PARAGMILK.NS', 'PATELENG.NS', 'PCJEWELLER.NS', 'PEL.NS', 'PENIND.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFC.NS', 'PFIZER.NS', 'PGEL.NS', 'PGHH.NS', 'PGIL.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PIIND.NS', 'PITTIENG.NS', 'PNB.NS', 'PNBHOUSING.NS', 'PNCINFRA.NS', 'POKARNA.NS', 'POLYCAB.NS', 'POLYMED.NS', 'POLYPLEX.NS', 'POWERGRID.NS', 'POWERINDIA.NS', 'POWERMECH.NS', 'PRAJIND.NS', 'PRAKASH.NS', 'PREMEXPLN.NS', 'PRESTIGE.NS', 'PRICOLLTD.NS', 'PRINCEPIPE.NS', 'PRIVISCL.NS', 'PSB.NS', 'PTC.NS', 'QUESS.NS', 'QUICKHEAL.NS', 'RADICO.NS', 'RAIN.NS', 'RAJESHEXPO.NS', 'RALLIS.NS', 'RAMCOCEM.NS', 'RATNAMANI.NS', 'RAYMOND.NS', 'RBLBANK.NS', 'RCF.NS', 'RECLTD.NS', 'REDINGTON.NS', 'REFEX.NS', 'RELIANCE.NS', 'RELIGARE.NS', 'RELINFRA.NS', 'RENUKA.NS', 'REPCOHOME.NS', 'RESPONIND.NS', 'RGL.NS', 'RIIL.NS', 'RITES.NS', 'RKFORGE.NS', 'ROHLTD.NS', 'ROUTE.NS', 'RPOWER.NS', 'RTNPOWER.NS', 'RVNL.NS', 'SAIL.NS', 'SALZERELEC.NS', 'SANGHVIMOV.NS', 'SANOFI.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SBIN.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SDBL.NS', 'SEQUENT.NS', 'SHAKTIPUMP.NS', 'SHALBY.NS', 'SHARDACROP.NS', 'SHARDAMOTR.NS', 'SHAREINDIA.NS', 'SHILPAMED.NS', 'SHK.NS', 'SHREECEM.NS', 'SHRIPISTON.NS', 'SIYSIL.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SKIPPER.NS', 'SMLISUZU.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SONATSOFTW.NS', 'SOUTHBANK.NS', 'SPAL.NS', 'SPANDANA.NS', 'SPARC.NS', 'SPMLINFRA.NS', 'SRF.NS', 'STAR.NS', 'STARCEMENT.NS', 'STLTECH.NS', 'SUBEXLTD.NS', 'SUDARSCHEM.NS', 'SUMICHEM.NS', 'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUNFLAG.NS', 'SUNPHARMA.NS', 'SUNTECK.NS', 'SUPREMEIND.NS', 'SURYAROSNI.NS', 'SUVEN.NS', 'SUVENPHAR.NS', 'SUZLON.NS', 'SWANENERGY.NS', 'SWARAJENG.NS', 'SWSOLAR.NS', 'SYMPHONY.NS', 'SYNGENE.NS', 'TAJGVK.NS', 'TANLA.NS', 'TARC.NS', 'TASTYBITE.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TBZ.NS', 'TCPLPACK.NS', 'TCS.NS', 'TDPOWERSYS.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEJASNET.NS', 'TEXINFRA.NS', 'TEXRAIL.NS', 'TFCILTD.NS', 'THANGAMAYL.NS', 'THEMISMED.NS', 'THERMAX.NS', 'THOMASCOOK.NS', 'TI.NS', 'TIINDIA.NS', 'TIMETECHNO.NS', 'TIMKEN.NS', 'TIRUMALCHM.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRITURBINE.NS', 'TRIVENI.NS', 'TTML.NS', 'TVSMOTOR.NS', 'UBL.NS', 'UCOBANK.NS', 'ULTRACEMCO.NS', 'UNICHEMLAB.NS', 'UNIONBANK.NS', 'UPL.NS', 'USHAMART.NS', 'UTIAMC.NS', 'V2RETAIL.NS', 'VADILALIND.NS', 'VAIBHAVGBL.NS', 'VAKRANGEE.NS', 'VALIANTORG.NS', 'VARROC.NS', 'VBL.NS', 'VEDL.NS', 'VESUVIUS.NS', 'VGUARD.NS', 'VIMTALABS.NS', 'VINATIORGA.NS', 'VINDHYATEL.NS', 'VIPCLOTHNG.NS', 'VIPIND.NS', 'VISHNU.NS', 'VMART.NS', 'VOLTAMP.NS', 'VOLTAS.NS', 'VSTIND.NS', 'VTL.NS', 'WABAG.NS', 'WALCHANNAG.NS', 'WEBELSOLAR.NS', 'WELCORP.NS', 'WELENT.NS', 'WHIRLPOOL.NS', 'WOCKPHARMA.NS', 'WSTCSTPAPR.NS', 'YESBANK.NS', 'ZEEL.NS', 'ZENSARTECH.NS', 'ZENTEC.NS', 'ACC.NS', 'ABSLAMC.NS']

start_date = "2023-05-01"
end_date = "2025-04-26"
interval = "1h"
batch_size = 50

# === CREATE FOLDER "hourly_data" IF IT DOESN'T EXIST ===
folder_name = "hourly_data"
os.makedirs(folder_name, exist_ok=True)

# === RETRY LOGIC FOR DOWNLOADING EACH TICKER ===
@retry(wait=wait_fixed(3), stop=stop_after_attempt(3))
def download_ticker(ticker):
    print(f"Downloading: {ticker}")
    df = yf.download(ticker, start=start_date, end=end_date, interval=interval, progress=False)
    return df

# === PROCESS IN BATCHES ===
for i in range(0, len(tickers), batch_size):
    batch = tickers[i:i + batch_size]
    print(f"\n📦 Processing batch {i // batch_size + 1} of {len(tickers) // batch_size + 1}")

    for ticker in batch:
        try:
            df = download_ticker(ticker)
            if not df.empty:
                file_path = os.path.join(folder_name, f"{ticker}_hourly.csv")
                df.to_csv(file_path)
            else:
                print(f"⚠️ No data for {ticker}")
        except Exception as e:
            print(f"❌ Error downloading {ticker}: {e}")
    
    # Optional sleep to reduce chance of rate-limiting
    time.sleep(5)

print(f"\n✅ Done! All CSV files saved in folder: {folder_name}")


📦 Processing batch 1 of 14
Downloading: 3MINDIA.NS
Downloading: 63MOONS.NS
Downloading: AARTIDRUGS.NS
Downloading: AARTIIND.NS
Downloading: AAVAS.NS
Downloading: ABB.NS
Downloading: ABBOTINDIA.NS
Downloading: ABCAPITAL.NS
Downloading: ABFRL.NS
Downloading: ACC.NS
Downloading: ACE.NS
Downloading: ADANIENT.NS
Downloading: ADANIGREEN.NS
Downloading: ADANIPORTS.NS
Downloading: ADANIPOWER.NS
Downloading: ADSL.NS
Downloading: ADVENZYMES.NS
Downloading: AFFLE.NS
Downloading: AHLUCONT.NS
Downloading: AIAENG.NS
Downloading: AJANTPHARM.NS
Downloading: AJMERA.NS
Downloading: ALEMBICLTD.NS
Downloading: ALKEM.NS
Downloading: ALKYLAMINE.NS
Downloading: ALLCARGO.NS
Downloading: ALOKINDS.NS
Downloading: AMBER.NS
Downloading: AMBUJACEM.NS
Downloading: ANANTRAJ.NS
Downloading: ANUP.NS
Downloading: APARINDS.NS
Downloading: APLAPOLLO.NS
Downloading: APLLTD.NS
Downloading: APOLLO.NS
Downloading: APOLLOHOSP.NS
Downloading: APOLLOTYRE.NS
Downloading: ARTEMISMED.NS
Downloading: ARVIND.NS
Downloading: ARVINDF